In [84]:
import numpy as np
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import gensim
from nltk.corpus import brown
import random
from sklearn.model_selection import KFold
import lightgbm as lgb
import gc
from keras.callbacks.callbacks import EarlyStopping
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

import keras
from keras.utils.vis_utils import plot_model
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Embedding, Flatten, LSTM, Dropout, Input, Conv1D, MaxPooling1D
from keras.layers.merge import concatenate
from keras.callbacks.callbacks import EarlyStopping
from keras import backend
from keras.preprocessing.sequence import pad_sequences
from scipy.stats import spearmanr
from nltk.corpus import wordnet as wn
import tqdm
from sklearn.model_selection import StratifiedKFold




In [85]:
train = pd.read_csv("../input/google-quest-challenge/train.csv")
test = pd.read_csv("../input/google-quest-challenge/test.csv")

In [86]:
sample_sub = pd.read_csv("../input/google-quest-challenge/sample_submission.csv")

In [87]:
train.shape

(6079, 41)

In [88]:
target_cols = ['question_asker_intent_understanding',
       'question_body_critical', 'question_conversational',
       'question_expect_short_answer', 'question_fact_seeking',
       'question_has_commonly_accepted_answer',
       'question_interestingness_others', 'question_interestingness_self',
       'question_multi_intent', 'question_not_really_a_question',
       'question_opinion_seeking', 'question_type_choice',
       'question_type_compare', 'question_type_consequence',
       'question_type_definition', 'question_type_entity',
       'question_type_instructions', 'question_type_procedure',
       'question_type_reason_explanation', 'question_type_spelling',
       'question_well_written', 'answer_helpful',
       'answer_level_of_information', 'answer_plausible', 'answer_relevance',
       'answer_satisfaction', 'answer_type_instructions',
       'answer_type_procedure', 'answer_type_reason_explanation',
       'answer_well_written']

In [89]:
train_labels = train[target_cols].values

In [90]:
len(train.columns)

41

In [91]:
len(target_cols)

30

# fearure engineering

In [92]:
def simple_prepro(s):
    return [w for w in s.replace("\n"," ").replace(","," , ").replace("("," ( ").replace(")"," ) ").
            replace("."," . ").replace("?"," ? ").replace(":"," : ").replace("n't"," not").
            replace("'ve"," have").replace("'re"," are").replace("'s"," is").split(" ") if w != ""]

In [93]:
def simple_prepro_tfidf(s):
    return " ".join([w for w in s.lower().replace("\n"," ").replace(","," , ").replace("("," ( ").replace(")"," ) ").
            replace("."," . ").replace("?"," ? ").replace(":"," : ").replace("n't"," not").
            replace("'ve"," have").replace("'re"," are").replace("'s"," is").split(" ") if w != ""])

In [94]:
qt_max = max([len(simple_prepro(l)) for l in list(train["question_title"].values)])
qb_max = max([len(simple_prepro(l))  for l in list(train["question_body"].values)])
an_max = max([len(simple_prepro(l))  for l in list(train["answer"].values)])
print("max lenght of question_title is",qt_max)
print("max lenght of question_body is",qb_max)
print("max lenght of question_answer is",an_max)

max lenght of question_title is 35
max lenght of question_body is 3404
max lenght of question_answer is 8179


In [95]:
qt_max = np.mean([len(simple_prepro(l)) for l in list(train["question_title"].values)])
qb_max = np.mean([len(simple_prepro(l))  for l in list(train["question_body"].values)])
an_max = np.mean([len(simple_prepro(l))  for l in list(train["answer"].values)])
print("mean lenght of question_title is",qt_max)
print("mean lenght of question_body is",qb_max)
print("mean lenght of question_answer is",an_max)

mean lenght of question_title is 9.991939463727586
mean lenght of question_body is 161.18703734166803
mean lenght of question_answer is 163.19608488238197


In [96]:
max_word = 35000
max_len = 1024
tokenizer = keras.preprocessing.text.Tokenizer(num_words=max_word)

In [97]:
train_drop = train[["question_title", "question_body","answer"]]
test_drop = test[["question_title", "question_body","answer"]]

In [98]:
test_drop

,question_title,question_body,answer
0,Will leaving corpses lying around upset my pri...,I see questions/information online about how t...,There is no consequence for leaving corpses an...
1,Url link to feature image in the portfolio,I am new to Wordpress. i have issue with Featu...,I think it is possible with custom fields.\n\n...
2,"Is accuracy, recoil or bullet spread affected ...","To experiment I started a bot game, toggled in...",You do not have armour in the screenshots. Thi...
3,Suddenly got an I/O error from my external HDD,I have used my Raspberry Pi as a torrent-serve...,Your Western Digital hard drive is disappearin...
4,Passenger Name - Flight Booking Passenger only...,I have bought Delhi-London return flights for ...,I called two persons who work for Saudia (tick...
...,...,...,...
471,change gb4e enumeration in only one chapter,I am using gb4e for glossing examples in my th...,You can simply reset the exx counter just afte...
472,All anagrams in a File,Source : Microsoft Interview Question\n\nWe ar...,Slightly different approach from the one above...
473,SQL Server: Time Series Table Design,I am creating a time series database that foll...,A foreign key can reference a unique constrain...
474,Is a countered flashback spell exiled?,If I flashback Past in Flames and it is counte...,If you counter a spell cast for its Flashback ...


In [99]:
data = train_drop.append(test_drop)

In [100]:
data.head()

,question_title,question_body,answer
0,What am I losing when using extension tubes in...,After playing around with macro photography on...,"I just got extension tubes, so here's the skin..."
1,What is the distinction between a city and a s...,I am trying to understand what kinds of places...,It might be helpful to look into the definitio...
2,Maximum protusion length for through-hole comp...,I'm working on a PCB that has through-hole com...,Do you even need grooves? We make several pro...
3,Can an affidavit be used in Beit Din?,"An affidavit, from what i understand, is basic...","Sending an ""affidavit"" it is a dispute between..."
4,How do you make a binary image in Photoshop?,I am trying to make a binary image. I want mor...,Check out Image Trace in Adobe Illustrator. \n...


In [101]:
for i in ["question_title", "question_body", "answer"]:
    tokenizer.fit_on_texts(data[i])    
print(len(tokenizer.word_index))

58158


In [102]:
sequences_title = tokenizer.texts_to_sequences(train["question_title"])
sequences_body = tokenizer.texts_to_sequences(train["question_body"])
sequences_answer = tokenizer.texts_to_sequences(train["answer"])
test_title = tokenizer.texts_to_sequences(test["question_title"])
test_body = tokenizer.texts_to_sequences(test["question_body"])
test_answer = tokenizer.texts_to_sequences(test["answer"])


In [103]:
title_pad = pad_sequences(sequences_title, maxlen=max_len)
body_pad =  pad_sequences(sequences_body, maxlen=max_len)
answer_pad =  pad_sequences(sequences_answer, maxlen=max_len)
tt_title_pad = pad_sequences(test_title, maxlen=max_len)
tt_body_pad =  pad_sequences(test_body, maxlen=max_len)
tt_answer_pad =  pad_sequences(test_answer, maxlen=max_len)

In [104]:
body_pad.shape

(6079, 1024)

In [105]:
train["category"].value_counts()

TECHNOLOGY       2441
STACKOVERFLOW    1253
CULTURE           963
SCIENCE           713
LIFE_ARTS         709
Name: category, dtype: int64

In [106]:
type2int = {type:i for i,type in enumerate(list(set(train["category"])))}
cate = np.identity(5)[np.array(train["category"].apply(lambda x:type2int[x]))].astype(np.float64)
cate_test = np.identity(5)[np.array(test["category"].apply(lambda x:type2int[x]))].astype(np.float64)

In [107]:
cate.shape

(6079, 5)

In [108]:
catego = np.zeros(title_pad.shape)
catego[:cate.shape[0],:cate.shape[1]] = cate

In [198]:
backend.clear_session()

In [191]:
from keras import backend as K
def correlation_coefficient_loss(y_true, y_pred):
    x = y_true
    y = y_pred
    mx = K.mean(x)
    my = K.mean(y)
    xm, ym = x-mx, y-my
    r_num = K.sum(tf.multiply(xm,ym))
    r_den = K.sqrt(tf.multiply(K.sum(K.square(xm)), K.sum(K.square(ym))))
    r = r_num / r_den

    r = K.maximum(K.minimum(r, 1.0), -1.0)
    return 1 - K.square(r)

In [199]:
def define_model(length, vocab_size):
	inputs1 = Input(shape=(length,))
	embedding1 = Embedding(vocab_size, 300)(inputs1)
	conv1 = Conv1D(filters=128, kernel_size=4, activation='relu')(embedding1)
	drop1 = Dropout(0.5)(conv1)
	pool1 = MaxPooling1D(pool_size=2)(drop1)
	flat1 = Flatten()(pool1)
	# channel 2
	inputs2 = Input(shape=(length,))
	embedding2 = Embedding(vocab_size, 300)(inputs2)
	conv2 = Conv1D(filters=128, kernel_size=4, activation='relu')(embedding2)
	drop2 = Dropout(0.5)(conv2)
	pool2 = MaxPooling1D(pool_size=2)(drop2)
	flat2 = Flatten()(pool2)
	# channel 3
	inputs3 = Input(shape=(length,))
	embedding3 = Embedding(vocab_size, 300)(inputs3)
	conv3 = Conv1D(filters=128, kernel_size=4, activation='relu')(embedding3)
	drop3 = Dropout(0.5)(conv3)
	pool3 = MaxPooling1D(pool_size=2)(drop3)
	flat3 = Flatten()(pool3)
	# merge
	merged = concatenate([flat1, flat2, flat3])
	# interpretation
	dense1 = Dense(64, activation='relu')(merged)
	outputs = Dense(len(target_cols), activation='sigmoid')(dense1)
	model = Model(inputs=[inputs1, inputs2, inputs3], outputs=outputs)
	# compile
	model.compile(loss='binary_crossentropy', optimizer='adam')
	# summarize
	print(model.summary())
	plot_model(model, show_shapes=True, to_file='multichannel.png')
	return model

In [200]:
model = define_model(1024, len(tokenizer.word_index))

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 1024)         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 1024)         0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 1024)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 1024, 300)    17447400    input_1[0][0]                    
____________________________________________________________________________________________

In [201]:
es = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=0, mode='auto', baseline=None, restore_best_weights=True)

hist = model.fit([title_pad,body_pad,answer_pad], np.array(train_labels), epochs = 5, validation_split=0.2, callbacks = [es])

/opt/conda/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 4863 samples, validate on 1216 samples
Epoch 1/5
4863/4863 [==============================] - 10s 2ms/step - loss: 0.4337 - val_loss: 0.4196
Epoch 2/5
4863/4863 [==============================] - 9s 2ms/step - loss: 0.3885 - val_loss: 0.4048
Epoch 3/5
4863/4863 [==============================] - 9s 2ms/step - loss: 0.3580 - val_loss: 0.3980
Epoch 4/5
4863/4863 [==============================] - 9s 2ms/step - loss: 0.3359 - val_loss: 0.3976
Epoch 5/5
4863/4863 [==============================] - 9s 2ms/step - loss: 0.3197 - val_loss: 0.4004


In [202]:
preds = model.predict([title_pad,body_pad,answer_pad])
val_y = train[target_cols]
overall_score = 0

In [203]:
for col_index, col in enumerate(target_cols):
    overall_score += spearmanr(preds[:, col_index], val_y[col].values).correlation/len(target_cols)

In [204]:
overall_score

0.5251801771532183

In [205]:
pd.DataFrame(model.predict([title_pad,body_pad,answer_pad]), columns=target_cols)

,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,question_not_really_a_question,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,0.919559,0.580834,0.408437,0.446054,0.075548,0.209167,0.615856,0.662164,0.035287,0.003770,...,0.838407,0.986968,0.551442,0.969967,0.988968,0.891765,0.879344,0.010328,0.340737,0.929613
1,0.865790,0.746418,0.008463,0.708313,0.997718,0.948156,0.566039,0.507949,0.666137,0.003379,...,0.882454,0.955097,0.659499,0.896337,0.891978,0.819570,0.023408,0.031812,0.711497,0.853361
2,0.900133,0.595616,0.005766,0.769905,0.879000,0.902773,0.627084,0.388069,0.217492,0.004152,...,0.705542,0.839462,0.662256,0.948267,0.971726,0.732290,0.478641,0.352122,0.597230,0.872209
3,0.870566,0.602197,0.281375,0.970349,0.972685,0.862790,0.537811,0.435293,0.095533,0.000372,...,0.846813,0.991382,0.585487,0.969043,0.955569,0.898575,0.091135,0.000342,0.965514,0.918161
4,0.841573,0.529979,0.002025,0.971657,0.813434,0.990474,0.648423,0.539771,0.046441,0.000807,...,0.700967,0.979858,0.701406,0.995390,0.996449,0.937041,0.944904,0.012869,0.818533,0.951753
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6074,0.916724,0.646196,0.088214,0.873593,0.817623,0.703247,0.582739,0.494780,0.435362,0.001938,...,0.827983,0.898398,0.700404,0.968843,0.954711,0.819388,0.037828,0.127161,0.896056,0.882993
6075,0.948292,0.638202,0.041788,0.416606,0.798532,0.561671,0.589788,0.491438,0.187388,0.004202,...,0.846479,0.850211,0.754220,0.958266,0.955068,0.774926,0.420178,0.568546,0.494354,0.832102
6076,0.819037,0.455294,0.002703,0.712424,0.831324,0.938888,0.562871,0.511966,0.086515,0.010856,...,0.635084,0.938896,0.585175,0.966252,0.979837,0.818431,0.940962,0.208071,0.248558,0.858192
6077,0.845510,0.608448,0.218047,0.493496,0.457683,0.560624,0.616075,0.580505,0.547150,0.009787,...,0.723534,0.872162,0.638828,0.934177,0.940028,0.768456,0.325074,0.103919,0.772877,0.864835


In [124]:
mission = pd.DataFrame(model.predict([tt_title_pad, tt_body_pad, tt_answer_pad]), columns=target_cols)

In [125]:
sub = pd.read_csv("../input/google-quest-challenge/sample_submission.csv")
sub = pd.merge(sub["qa_id"], mission, left_index=True, right_index= True)
sub.to_csv("submission.csv", index = False)